In [1]:
#| default_exp process

In [2]:
#| hide
from nbdev.showdoc import *
from nbdev import *

In [3]:
#| export
import configparser
from fastcore.basics import patch
from os.path import expanduser
from astroquery.astrometry_net import AstrometryNet
from astroquery.exceptions import TimeoutError as ASTTimeoutError
from osob.core import Telescope
from osob.solver import Solver
from tqdm.auto import tqdm

In [4]:
#| local
config = configparser.ConfigParser()
config.read(expanduser('~/.config/telescope.ini'))

['/home/jochym/.config/telescope.ini']

In [5]:
#| local
OSO=Telescope(config['telescope.org']['user'], 
              config['telescope.org']['password'])

In [6]:
#| local
def get_object_obs(obj):
    reqlst=OSO.get_user_requests(sort='completion')
    print(f'Number of users requests: {len(reqlst)}')

    complete = [rq for rq in sorted(reqlst, key=lambda r: int(r['requesttime']), reverse=True) 
                        if Telescope.REQUESTSTATUS_TEXTS[int(rq['status'])]=='Complete']
    print('Completed:', len(complete))
    objjobs = ((int(OSO.get_request(int(j['id']))['jid'][1:]), int(j['id'])) 
               for j in complete if j['objectname']==obj)
    return objjobs

In [7]:
reqlst=OSO.get_user_requests(sort='completion')
complete = [rq for rq in sorted(reqlst, key=lambda r: int(r['requesttime']), reverse=True) 
                        if Telescope.REQUESTSTATUS_TEXTS[int(rq['status'])]=='Complete']

In [8]:
for rq in tqdm(complete):
    name = rq['objectname']
    rid = int(rq['id'])
    try:
        jid = int(OSO.get_request(rid)['jid'][1:])
        data = OSO.get_obs(OSO.get_job(jid), cube=True, verbose=False)
        if data :
            fp = f'/home/jochym/Astro/VS/{"_".join(name.split())}'
            os.makedirs(fp, exist_ok=True)
            fn = f'{jid}'
            try :
                os.symlink(os.path.abspath(data.name), f'{fp}/{fn}.fits')
            except FileExistsError:
                pass
        else :
            print(f'Download of J{jid} failed')
    except TimeoutError :
        print(f'Download of J{jid} failed')
        pass

  0%|          | 0/1177 [00:00<?, ?it/s]

Download of J321992 failed
Download of J321989 failed
Download of J294696 failed
Download of J293658 failed
Download of J293657 failed
Download of J293656 failed
Download of J293655 failed
Download of J293654 failed
Download of J293653 failed
Download of J293652 failed
Download of J293651 failed


In [29]:
VS = 'DQ Vul'
jlist = list(get_object_obs(VS))

for jid, rid in tqdm(jlist):
    #print(f"R{rid}:J{jid}")
    try :
        data = OSO.get_obs(OSO.get_job(jid), cube=True, verbose=False)
        if data :
            with open(f'/home/jochym/Astro/VS/{"_".join(VS.split())}/{jid}.fits', 'wb') as ff:
                ff.write(data.read())
        else :
            print(f'Download of J{jid} failed')
    except TimeoutError:
        print(f'Download of J{jid} failed')
        continue

Number of users requests: 1200
Completed: 1177


  0%|          | 0/94 [00:00<?, ?it/s]

In [47]:
os.path.abspath(OSO.cache)

'/home/jochym/Projects/osob/.cache/jobs'

In [42]:
!ls -l /home/jochym/Astro/VS/V1223_Sgr/

ls: nie ma dostępu do '/home/jochym/Astro/VS/V1223_Sgr/': Nie ma takiego pliku ani katalogu


In [48]:
os.path.abspath(data.name)

'/home/jochym/Projects/osob/.cache/jobs/3/4/340922.fits'